In [101]:
import requests
import pdf2image 
import numpy as np
from PIL import Image
import pytesseract
import re
from IPython.display import display
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import unidecode
import io
import PyPDF2
import bs4 as bs
import os
import pandas as pd
import glob
from dotenv import load_dotenv
import asyncio
from tqdm import tqdm
import asyncio
import aiohttp
import aiofiles

In [84]:
!pip3 install aiofiles


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [76]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [70]:
def get_dataframe(folder):
    data = np.array(sorted([(int(alpha[0]),int(alpha[1]), x)  for x in glob.glob(f'{folder}/*.jpg') if\
                   (alpha:=os.path.basename(x).replace('jpg', '')[:-1].split('_'))]))

    df= pd.DataFrame(data=data, columns = ['Chapter', 'Formula#', 'Path'])
    df['latex'] = None
    return df

In [71]:
principles = get_dataframe('principles')

In [78]:
principles

,Chapter,Formula#,Path,latex
0,1,2,principles/1_2.jpg,None
1,1,3,principles/1_3.jpg,None
2,1,4,principles/1_4.jpg,None
3,1,5,principles/1_5.jpg,None
4,2,1,principles/2_1.jpg,None
...,...,...,...,...
596,11,105,principles/11_105.jpg,None
597,11,106,principles/11_106.jpg,None
598,11,107,principles/11_107.jpg,None
599,11,108,principles/11_108.jpg,None


In [144]:
load_dotenv()
APP_ID = os.getenv('APP_ID')
APP_KEY = os.getenv('APP_KEY')

pattern_2 = r"(\(\d+\))"
compiled_pattern_2 = re.compile(pattern_2)

headers = {
        'app_id': APP_ID,
        'app_key': APP_KEY,
    }

def get_latex(path_to_image):
    global headers

    files = { 
        'file': open(path_to_image,"rb"),
        'options_json': (None, json.dumps({'math_inline_delimiters': ['$', '$'], 'rm_spaces': True, 'ocr': ['math'], 'formats': ['text']})),
    }
    try:
        response = requests.post('https://api.mathpix.com/v3/text', headers=headers, files=files)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        exit(1) 

    try:
        latex_response = json.loads(response.text)['text']
    except:
        latex_response = response

    try:
        latex_response = latex_response.replace(re.findall(compiled_pattern_2, latex_response)[0], '').strip()
    except:
        latex_response = None
    
    return latex_response


tqdm.pandas()

principles['latex'] = principles['Path'].progress_apply(get_latex)

100%|█████████████████████████████████████████| 601/601 [01:05<00:00,  9.22it/s]


In [157]:
os.chdir(os.getcwd())

In [172]:
path = os.path.join(os.path.split(os.getcwd())[0], 'training_data', '')

In [179]:
principles.to_csv(f'{path}/principles_formulas.csv', index = None)

In [182]:
test_df = pd.read_csv(f'{path}/principles_formulas.csv')

In [183]:
test_df

,Chapter,Formula#,Path,latex
0,1,2,principles/1_2.jpg,\[\nm^{2}=2 n^{2}\n\]
1,1,3,principles/1_3.jpg,\[\nq=p-\frac{p^{2}-2}{p+2}=\frac{2 p+2}{p+2}\n\]
2,1,4,principles/1_4.jpg,\[\nq^{2}-2=\frac{2\left(p^{2}-2\right)}{(p+2)...
3,1,5,principles/1_5.jpg,$$\n\[\nn_{0}+\frac{n_{1}}{10}+\cdots+\frac{n_...
4,2,1,principles/2_1.jpg,\[\nS=\bigcup_{\alpha \in A} E_{\alpha} .\n\]
...,...,...,...,...
596,11,105,principles/11_105.jpg,$$\n\[\n\left\|f-f_{n}\right\| \leq\left\|f-f_...
597,11,106,principles/11_106.jpg,\[\nf \sim \sum_{n=1}^{\infty} c_{n} \phi_{n}\n\]
598,11,107,principles/11_107.jpg,$$\n\[\n\int_{X}|f|^{2} d \mu=\sum_{n=1}^{\inf...
599,11,108,principles/11_108.jpg,"\[\ng \sim \sum_{n=1}^{\infty} c_{n} \phi_{n},..."
